https://arxiv.org/abs/1711.05772 / https://arxiv.org/abs/1802.04877

https://github.com/natashamjaques/magenta/blob/affective-reward/magenta/models/affective_reward/latent_gan.py

## Labeling data

In [1]:
#from pdb import set_trace
#set_trace()

In [2]:
import numpy as np

labels = np.empty([0, 2], dtype=int)
gens_keep = np.empty([0, ], dtype=object)
zs_keep = np.empty([0, 1, 128], dtype=float)

In [3]:
from pathlib import Path
vae_dir = Path('../../pytorch-text-vae') # https://github.com/iconix/pytorch-text-vae

n_samples = 100
temp = 0.2

%run -i $vae_dir/generate.py $vae_dir/reviews_sentences_state.pt reviews_sentences_stored_info.pkl --cache-path=$vae_dir/tmp --num-sample=$n_samples --temp=$temp

Namespace(cache_path='../../pytorch-text-vae/tmp', max_length=50, num_sample=100, print_z=False, saved_vae='../../pytorch-text-vae/reviews_sentences_state.pt', seed=None, stored_info='reviews_sentences_stored_info.pkl', temp=0.2, use_cuda=True)
Fetching cached info at ../../pytorch-text-vae/tmp/reviews_sentences_stored_info.pkl
Cache ../../pytorch-text-vae/tmp/reviews_sentences_stored_info.pkl loaded, total load time 0.6427538394927979
Found saved model ../../pytorch-text-vae/reviews_sentences_state.pt
MAX_SAMPLE: False; TRUNCATED_SAMPLE: True
Setting new random seed
---
it came out in the kind my life and it along with him
---
take as a UNK at part of the house vocals are so perfection
---
inspired by the vocals and make trying to give this one accompanied by them as they instrumentation
---
this is taken from the mash up like this is one of this though
---
over the link below and are up over the dose of artist good link below
---
to take on a brand new single from from and she takes 

---
and we should even part of the track what you should even better
---
and according to this UNK was inspired with the title of its really lines
---
UNK aka artist has a stripped back in the sound of the sound and the sound electro perfection
---
just as one of acoustic UNK and and just featured in the UNK take on to date
---
in UNK UNK theres a new cut from your below and rather than style


In [4]:
list(zip(range(len(gens)), gens))

[(0, 'it came out in the kind my life and it along with him'),
 (1, 'take as a UNK at part of the house vocals are so perfection'),
 (2,
  'inspired by the vocals and make trying to give this one accompanied by them as they instrumentation'),
 (3, 'this is taken from the mash up like this is one of this though'),
 (4, 'over the link below and are up over the dose of artist good link below'),
 (5, 'to take on a brand new single from from and she takes on a four times'),
 (6,
  'UNK UNK UNK UNK even if youre listening to the likes of one and if you not her attention'),
 (7,
  'its one of the electro pop sound that has away from the forthcoming full sound'),
 (8,
  'continues to returns from his debut from last week and i want to create a one of electro house style'),
 (9,
  'like some of artist continues to UNK and serves as one of the this year my recordings'),
 (10, 'UNK will be taken from the likes to ep in march and ive been luna'),
 (11, 'and how they take on this UNK and a perfect 

In [5]:
#new_labels = np.array([(1, -1), (10, -1)])

# -1 is "bad" because it contains a banned word, 1 is "good"
banned = ['feat']
labels = np.ones(n_samples, dtype=int)
gens_lose = set([i for b in banned for i in np.where([b in g.split() for g in gens])[0]])
gens_keep = list(set(range(len(gens))) - gens_lose)
labels[gens_keep] = -1
zs_keep = zs

labels

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,
        1, -1, -1,  1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [6]:
#labels = np.concatenate((labels, new_labels))
#gens_keep = np.concatenate((gens_keep, np.array(gens)[np.array([i for i, l in new_labels])]))
#zs_keep = np.concatenate((zs_keep, np.array([z.cpu().numpy() for z in zs])[np.array([i for i, l in new_labels])]))

#gens_keep

In [7]:
#gens_keep.shape, zs_keep.shape, labels.shape

In [8]:
#np.get_printoptions()

In [9]:
#np.set_printoptions(threshold=np.inf)
#zs_keep

In [10]:
# -1 is "bad", 1 is "good"
#labels = np.ones(n_samples)

# for now, make up some random labels
#np.random.seed(42)
#ones = np.random.rand(n_samples) < 0.2
#labels[ones] = -1

#labels = np.array([l for i, l in labels], dtype=float)
#labels

In [11]:
from collections import Counter
c1 = Counter([word for gen in gens for word in gen.split()])
[(b, c1[b]) for b in banned], c1.most_common(25)

([('feat', 9)],
 [('the', 99),
  ('and', 81),
  ('UNK', 77),
  ('to', 69),
  ('a', 65),
  ('of', 64),
  ('artist', 33),
  ('from', 28),
  ('with', 24),
  ('in', 22),
  ('been', 20),
  ('as', 19),
  ('trying', 18),
  ('on', 18),
  ('has', 18),
  ('this', 17),
  ('is', 17),
  ('it', 16),
  ('one', 16),
  ('be', 16),
  ('new', 15),
  ('that', 14),
  ('track', 14),
  ('for', 13),
  ('up', 12)])

## Data

In [12]:
batch_size = 16

In [13]:
from fastai.dataset import *

class LatentDataset(Dataset):
    def __init__(self, x, y): self.x,self.y = x,y
    def __getitem__(self, idx): return A(self.x[idx], self.y[idx])
    def __len__(self): return len(self.x)
    
ds = LatentDataset(zs_keep, labels.astype(float))
dl = DataLoader(ds, batch_size)
md = ModelData('.', dl, None)

In [14]:
md.trn_ds[0]

[array([[ 0.03062, -0.40484,  0.14326,  1.03329, -0.61295,  1.28334, -0.34915, -0.68755,  0.63758,  0.46367,
          0.14551, -0.37912,  0.41953,  0.32616, -0.2933 , -0.60283,  0.05167, -0.68704,  0.08463,  0.20438,
         -0.95775, -0.08818,  0.03897, -0.35505,  1.28753, -0.7905 , -0.509  , -0.89471, -1.38843, -0.82167,
          0.10482, -0.74657, -0.19068, -1.02   ,  0.00734,  1.3949 , -1.09858, -0.61614, -0.44469,  0.52185,
          0.8835 ,  1.28144, -0.04392, -1.66312, -0.20609, -0.23686,  1.19621, -0.09359,  0.37392,  0.7714 ,
         -0.79537,  0.79677, -0.89384, -0.15601,  0.49538, -0.11979, -0.27753, -0.00265,  0.28338,  0.76306,
         -1.5272 ,  0.26166,  1.46766,  0.95444, -0.55457, -1.38998, -0.94134, -0.42574,  0.80968, -0.54563,
         -0.617  ,  0.07215, -0.86152,  2.08143, -0.04659,  0.29063,  0.01217,  0.28925, -0.79423,  0.87405,
          1.2639 , -0.63003,  1.04756,  1.20437, -0.575  ,  1.34179,  0.88682,  0.10663,  0.31128,  0.45204,
         -0.49103, 

## Model

In [15]:
n_latent = 128
n_hidden = 1024
lr = 3e-4

In [16]:
import torch.optim as optim
import torch.nn as nn

class LCGAN_D(nn.Module):
    '''Discriminator'''
    def __init__(self, n_latent, n_hidden=n_hidden, n_output=1):
        super(LCGAN_D, self).__init__()
        
        self.i2h = nn.Linear(n_latent, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        self.h2o = nn.Linear(n_hidden, n_output)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, z):
        x = z
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        v = self.sigmoid(self.h2o(x))
        
        return v

class LCGAN_G(nn.Module):
    '''Generator'''
    def __init__(self, n_latent, n_hidden=n_hidden):
        super(LCGAN_G, self).__init__()
        self.n_latent = n_latent
        
        self.i2h = nn.Linear(n_latent, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        # hidden-to-gating mechanism
        self.h2g = nn.Linear(n_hidden, 2*n_latent)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, z):
        x = z
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        x = self.h2g(x)
        
        # gating mechanism: allow network to remember/forget
        # what it wants to about the original z and x
        z_mid = x[:, self.n_latent:]
        gates = self.sigmoid(x[:, :self.n_latent])
        dz = gates * z_mid # TODO: why naming?
        z_prime = (1 - gates)*z + dz
        
        return z_prime

In [17]:
class LCGAN(nn.Module):
    def __init__(self, D, G, batch_size=batch_size):
        super(LCGAN, self).__init__()
        self.batch_size = batch_size
        
        self.D = D
        self.G = G

    def train(self):
        self.D.train()
        self.G.train()
        
    def forward(self, z=None):       
        if z is not None:
            # train discriminator
            v = self.D(z)
            return v
        else: # train GAN
            # gaussian random noise
            z_prior = torch.randn(self.batch_size, self.G.n_latent).cuda()
            
            z_prime = self.G(z_prior)
            v_prime = self.D(z_prime)
            
            return v_prime

## Training

In [18]:
gan = LCGAN(LCGAN_D(n_latent), LCGAN_G(n_latent)).cuda()

In [19]:
import fastai as fast

fast.core.set_trainable(gan.D, True)
fast.core.set_trainable(gan.G, True)

opt_d = optim.Adam(gan.D.parameters(), lr=lr)
opt_g = optim.Adam(gan.G.parameters(), lr=lr)

In [20]:
# test what the GAN is doing before any training

def generate(z):
    generated = vae.decoder.generate(z, 50, temp, True)
    generated_str = model.float_word_tensor_to_string(output_side, generated)

    EOS_str = f' {output_side.index_to_word(torch.LongTensor([EOS_token]))} '

    if generated_str.endswith(EOS_str):
        generated_str = generated_str[:-5]

    # flip it back
    return generated_str[::-1]
    
generate(gan.G(torch.randn(1, n_latent).cuda()))

'along on a new york based producer UNK has managed to keep your musical feat'

In [21]:
for i in range(10):
    print(generate(gan.G(torch.randn(1, n_latent).cuda())))

remix of a track that continues to put it in the sort of stripped here at times
the comprised of UNK and serves as a stuck in better known as though
this UNK is a going to comes in in UNK and no feat
as one of the UNK i listened to a new single from white feat
and this be one of the year when they look at times
as a remix of UNK and gearing up at the months where we were fire
the UNK will take on part of no matter if i look at feat
inspired by the UNK remix of artist has a manages to to fall in los times
and serves as one of the UNK and a their trying to look at times
artist are back a UNK and the part of you from their previous chance


In [22]:
#from tqdm import tqdm

# adapted from: https://github.com/fastai/fastai/blob/master/courses/dl2/wgan.ipynb
def train(n_iter, alternate=False, first=False):
    gen_iters = 0
    for epoch in trange(n_iter):
        gan.train()
        data_iter = iter(md.trn_dl)
        i, n = 0, len(md.trn_dl)
        
        def train_G():
            ''' Train generator '''
            nonlocal gen_iters
            
            fast.core.set_trainable(gan.D, False)
            fast.core.set_trainable(gan.G, True)

            gan.G.zero_grad()

            #print(i, n)
            loss_g = - torch.log(gan()).mean()
            loss_g.backward()
            opt_g.step()
            gen_iters += 1
            
            return loss_g

        def train_D():
            ''' Train discriminator '''
            nonlocal i
            
            fast.core.set_trainable(gan.D, True)
            fast.core.set_trainable(gan.G, False)
            d_iters = 100 if (first and (gen_iters < 25) or (gen_iters % 500 == 0)) else 3
            j = 0

            while (j < d_iters) and (i < n):
                j += 1; i += 1
                batch = next(data_iter)
                #print(j, i, batch[0].size(), batch[1].size())
                z_real = batch[0].cuda()
                v = gan(z_real)

                gan.D.zero_grad()

                loss_d = - (batch[1] * torch.log(v) + (1.0-batch[1]) * torch.log(1.0 - v)).mean()
                loss_d.backward()
                opt_d.step()
                
                pbar.update()
                
            return loss_d
        
        with tqdm(total=n) as pbar:
            while i < n:
                if alternate:
                    # train discriminator
                    loss_d = train_D()
                    # then train generator a little bit
                    loss_g = train_G()
                else:
                    # train generator only
                    i += 1
                    loss_g = train_G()
                    pbar.update()
        
        if alternate:
            print(f'Loss_D {to_np(loss_d)}; Loss_G {to_np(loss_g)}; ')
        else:
            print(f'Loss_G {to_np(loss_g)}; ')

In [23]:
def train_and_generate(gan, n_epoch, alternate=False, n_sample=10):
    train(n_epoch, alternate)
    for i in range(n_sample):
        print(generate(gan.G(torch.randn(1, n_latent).cuda())))

In [24]:
train_and_generate(gan, 1, alternate=True)

100%|██████████| 7/7 [00:00<00:00, 239.63it/s]
Loss_D -3.456084966659546; Loss_G 2.534956932067871; 
he continues to chilled one of the year and it gets really away his together
in UNK and serves as one of the trying to look into no times
in the artist have teamed up with a brand new single his feat
single from artist and serves as one of of emotion and head look at times
artist is back to a full of synths and stuck in this one you date
artist is continues to come along with a remix of UNK who has been waiting for my times
artist is gearing up with a one of the track that has manages to pure alone
artist has teamed up with a nu a groove that stuck in the better known for artist live sitename
i feel with the rest of the UNK is back with a fresh and stuck in my tour
she has teamed up with a new UNK who has turned it in the one of my perfection


In [25]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 426.52it/s]
Loss_G 1.8733115196228027; 
a sort of electro pop aka artist has been known for the electro pop banger
one of the artist and such a brand new single from most creative scene
artist has teamed up released a new part of the UNK which comes along with an synth line
with a bit to UNK UNK and the UNK is one of your real scene
listen to UNK be such as part of the track is taken from its most recordings
UNK is taken from the likes of german aka artist such an amazing banger
a bit a indie pop aka artist is continues to along with the electro pop vibes
the indie pop track UNK is taken from the scene along with a very pop line
artist has teamed up with a new remix of the track will is known for an no feat
this UNK is taken of the UNK and and such a very perfection


In [26]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 438.85it/s]
Loss_G 1.3772995471954346; 
with this UNK in the track to be such as one under my feat
artist has teams up with the one of my year and which is taken from his live times
the pop track that seems to be featured in the UNK long as part of their solo feat
UNK and serves as one of the track which perfect trying i look many times
to UNK and all in the UNK with a even my rather my feat
artist is back to a long as part of the track comes collaboration with its no feat
artist has been continues to come along in the sort of trying to give it into an your times
this UNK and UNK has a bit of and of the track theres no times
to a new track called below and serves is one of the stuck on pretty feat
as one of the UNK trying to give it of a portland no feat


In [27]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 424.73it/s]
Loss_G 1.0681360960006714; 
artist manages to come up with a UNK and according to the song so few times
part of of UNK who has teamed up with a brand new its coming times
artist seems to to serves as one of the music and rather than every feat
of pop track that he seems to put one of the UNK year and its many times
of a brand new track but it seems to be taken of the UNK and its many feat
but it seems to be taken from the UNK and theres no feat
artist artist continues to serves as one of the melody the stuck in my times
artist is a listen to the side of what i look at your times
artist has up with a UNK and listen to the one of the and theres no feat
it would is taken from the UNK and was trying to keep your feat


In [28]:
g2 = []

for i in range(100):
    g2.append(generate(gan.G(torch.randn(1, n_latent).cuda())))
    
c2 = Counter([word for gen in g2 for word in gen.split()])
[(b, c2[b]) for b in banned], c2.most_common(25)

([('feat', 58)],
 [('and', 107),
  ('the', 102),
  ('of', 90),
  ('to', 83),
  ('UNK', 81),
  ('a', 71),
  ('artist', 62),
  ('feat', 58),
  ('one', 54),
  ('is', 43),
  ('times', 42),
  ('as', 39),
  ('with', 35),
  ('no', 32),
  ('from', 30),
  ('trying', 28),
  ('serves', 28),
  ('your', 25),
  ('track', 24),
  ('has', 23),
  ('taken', 23),
  ('many', 21),
  ('up', 21),
  ('every', 17),
  ('continues', 16)])

In [29]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 416.34it/s]
Loss_G 1.0038490295410156; 
later this UNK who and coming out in the better known as no times
a new single from UNK and listen to the remix of long as long so many times
as he continues a UNK as part of the next year for many times
with the remix the UNK UNK will take as a listen because its no times
from the UNK and serves as one of was trying to keep your no times
he along with a mash UNK and reminiscent of the track track theres no times
artist continues and a reminiscent of a track that will take from their real times
artist is from the likes the UNK and according to a long for many times
artist continues to come along with the sort of trying to give it into an many times
as a listen to UNK and UNK as one to the better ready for many times


In [30]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 418.21it/s]
Loss_G 0.966079592704773; 
serves as one of the UNK and is trying to keep your no feat
with a new remix of UNK and it seems to be taken from the UNK theres no feat
at the likes of UNK and listening to a brand the UNK because theres no feat
artist has been up with as one of the UNK and it would be taken for many feat
the new track is called UNK and according to the one of the UNK theres no feat
artist with a remix a UNK and in according of the UNK from your no feat
artist continues to serves as one of the UNK and i it into an no feat
a pop track where he continues to turned part of the UNK and rather than no feat
artist has teams up with a one of the UNK and it would be taken for from at feat
UNK and serves has a listen to a one of the UNK theres no feat


In [31]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 432.65it/s]
Loss_G 0.9576628804206848; 
from the UNK and serves as a one is trying to keep for many times
with a UNK and serves as one of of trying to give you under my times
a new track called UNK and serves as one of the trying to look theres no times
artist has been managed to take as one of the UNK theres no feat
with a called UNK and and seems to and one of the next year for many times
track is taken from the UNK and serves as one and its trying to keep your times
artist has been according to the UNK and put out of a few years for many times
artist is continues to serves as one of new UNK track theres no times
artist is continues to UNK and a part of the more track theres no times
artist is grew up and a part of their debut track because theres no times


In [32]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 424.06it/s]
Loss_G 0.950273871421814; 
one of serves with a UNK and listening to the music because theres many feat
artist has teams up with a UNK and one of the fell in san where no feat
artist has continues to UNK and serves as one of the UNK theres no feat
with a UNK and serves as one of of trying to check year for many feat
be blown away to UNK as one and the better known keep your no feat
a remix of even but it to be taken of the UNK and theres many feat
serves as one of the UNK and was trying to keep your no times
UNK is taken of the UNK and is trying to keep your no feat
a new single called UNK and serves as one of the trying to look to no feat
where he continues to serves as one of the UNK and theres many times


In [33]:
train_and_generate(gan, 1)

100%|██████████| 7/7 [00:00<00:00, 427.83it/s]
Loss_G 0.9456344842910767; 
artist continues to serves as one of the UNK and give track at no feat
artist has teams up with a UNK and one of the the trying would be known for many feat
artist is trying to and one of their new track theres no feat
he up with called UNK recorded in the one is trying to keep your no feat
UNK is one of a deep house where i to know for many feat
soon as serves with a UNK and according to the it under real times
with a new remix of UNK artist has been listening to the electro pop UNK theres no feat
artist continues to last UNK serves and one of the track and theres many feat
artist has teamed up with a a one of the UNK which was trying to keep his many feat
artist continues to serves with of UNK and stuck to the better known for many feat


In [34]:
train_and_generate(gan, 5)

100%|██████████| 7/7 [00:00<00:00, 443.10it/s]
Loss_G 0.9434221982955933; 
100%|██████████| 7/7 [00:00<00:00, 467.93it/s]
Loss_G 0.9410950541496277; 
100%|██████████| 7/7 [00:00<00:00, 469.66it/s]
Loss_G 0.939498782157898; 
100%|██████████| 7/7 [00:00<00:00, 467.91it/s]
Loss_G 0.9392887949943542; 
100%|██████████| 7/7 [00:00<00:00, 470.59it/s]
Loss_G 0.9369213581085205; 
artist is continues to to a bit of and of the UNK theres no feat
up with a one of the UNK which is trying to keep your no feat
with a UNK UNK and he trying to be one of the track because theres many feat
a a new track called UNK and serves as one of the perfect trying to look many feat
listening to a new track called UNK and he take as one of the track theres no feat
part of what he manages to UNK to more taken of the UNK theres no feat
artist has teams up with a UNK and one of the such as long as no feat
artist is back to a long as part of the track be taken from your no feat
UNK is taken from the UNK and one of a was

In [35]:
train_and_generate(gan, 5)

100%|██████████| 7/7 [00:00<00:00, 442.34it/s]
Loss_G 0.9382340312004089; 
100%|██████████| 7/7 [00:00<00:00, 477.04it/s]
Loss_G 0.9350211024284363; 
100%|██████████| 7/7 [00:00<00:00, 468.39it/s]
Loss_G 0.9355571866035461; 
100%|██████████| 7/7 [00:00<00:00, 467.99it/s]
Loss_G 0.9368972182273865; 
100%|██████████| 7/7 [00:00<00:00, 471.38it/s]
Loss_G 0.9335988759994507; 
artist is back be to long as part of the UNK and rather than no times
track be taken to the UNK and serves as one of his many times
artist is back to a long a one of the what i look at my feat
artist has continues a mash UNK serves as one of the UNK theres no feat
the new UNK artist a mash up and as one from their portland no feat
he continues to long as one of the UNK and make such a many feat
from the likes of UNK and listened to a brand new taken from what no times
artist is back to a long as one of the UNK theres no feat
UNK has a listen to the side of such as long as no feat
artist has been listen to the UNK and 

In [36]:
g3 = []

for i in range(100):
    g3.append(generate(gan.G(torch.randn(1, n_latent).cuda())))
    
c3 = Counter([word for gen in g3 for word in gen.split()])
[(b, c3[b]) for b in banned], c3.most_common(25)

([('feat', 49)],
 [('UNK', 114),
  ('the', 111),
  ('of', 102),
  ('to', 83),
  ('and', 78),
  ('a', 69),
  ('times', 51),
  ('one', 49),
  ('feat', 49),
  ('many', 46),
  ('artist', 44),
  ('no', 43),
  ('with', 35),
  ('as', 35),
  ('trying', 35),
  ('is', 32),
  ('taken', 30),
  ('from', 25),
  ('has', 23),
  ('track', 23),
  ('it', 21),
  ('be', 20),
  ('serves', 18),
  ('theres', 18),
  ('for', 18)])